**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
from scripts.postprocessing import *
from scripts.protein_preparation import *

[00:32:06] Initializing Normalizer
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/matplotlib/__init__.py:152: DeprecationWar

In [2]:
software = Path('/home/tony/DockM8/software')
receptor = Path('/home/tony/DockM8/testing_single_docking/protein.pdb')
ref_file = Path('/home/tony/DockM8/testing_single_docking/ref.sdf')
docking_library = Path('/home/tony/DockM8/testing_single_docking/library.sdf')
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS', 'bestpose_QVINAW', 'bestpose_QVINA2']
rescoring_functions = ['GNINA_Affinity','CNN-Score','CNN-Affinity', 'AD4', 'CHEMPLP', 'RFScoreVS', 'LinF9', 'SCORCH', 'Vinardo', 'PLECScore', 'NNScore', 'KORPL', 'ConvexPLR', 'AAScore']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'GypsumDL'
ncpus = 5
pocket = 'reference'
prepare_protein = 1
open('log.txt', 'w').close()


In [3]:
if prepare_protein == 1:
    #Prepare the protein
    prepared_receptor = prepare_protein_protoss(receptor)
else:
    prepared_receptor = receptor

#Create a temporary folder for all further calculations
w_dir = prepared_receptor.parent / prepared_receptor.stem
print('The working directory has been set to:', w_dir)
(w_dir).mkdir(exist_ok=True)

if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, prepared_receptor, 8)
    print(pocket_definition)
if pocket == 'RoG':
    pocket_definition = get_pocket_RoG(ref_file, prepared_receptor)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(prepared_receptor, w_dir, method='volume')
    print(pocket_definition)


[2023-Nov-21 00:32:10]: Preparing protein with ProtoSS ...
Job 2f7717e1-2782-41dc-af34-a32d7a9f1099 completed with success
The working directory has been set to: /home/tony/DockM8/testing_single_docking/protein_protoss

[2023-Nov-21 00:32:12]: Extracting pocket from protein_protoss using ref as reference ligand

[2023-Nov-21 00:32:28]: Finished extracting pocket from protein_protoss using ref as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [16.0, 16.0, 16.0]}


In [ ]:
if (w_dir / 'final_library.sdf').is_file() == False:
    prepare_library(docking_library, w_dir, id_column, protonation, software, ncpus)

In [ ]:
docking(w_dir, prepared_receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5, 'multiprocessing')

In [ ]:
concat_all_poses(w_dir, docking_programs, prepared_receptor)

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(str(w_dir / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
print(f'Loaded {len(all_poses)} poses.')
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


In [ ]:
for metric in clustering_metrics:
    if os.path.isfile(w_dir / f'clustering/{metric}_clustered.sdf') == False:
        filtered_poses = cluster_pebble(metric, 'KMedoids', w_dir, prepared_receptor, pocket_definition, software, all_poses, ncpus)

In [4]:
for metric in clustering_metrics:
        rescore_poses(w_dir, prepared_receptor, pocket_definition, software, w_dir / 'clustering' / f'{metric}_clustered.sdf', rescoring_functions , ncpus)



[2023-Nov-21 00:32:28]: Rescoring with SCORCH

[2023-Nov-21 00:32:46]: Rescoring with SCORCH complete in 18.1101!

[2023-Nov-21 00:32:46]: Skipping functions: GNINA_Affinity, CNN-Score, CNN-Affinity, AD4, CHEMPLP, RFScoreVS, LinF9, Vinardo, PLECScore, NNScore, KORPL, ConvexPLR, AAScore

[2023-Nov-21 00:32:46]: Combining all scores for /home/tony/DockM8/testing_single_docking/protein_protoss/rescoring_RMSD_clustered


Combining scores: 100%|██████████| 13/13 [00:00<00:00, 407.88files/s]



[2023-Nov-21 00:32:46]: Rescoring complete in 18.1810!


100%|██████████| 8/8 [00:00<00:00, 431.60it/s]


[2023-Nov-21 00:32:46]: Split docking library into 8 files each containing 4 compounds


...


[2023-Nov-21 01:44:11]: Split docking library into 10 files each containing 10 compounds



100%|██████████| 10/10 [07:58<00:00, 47.85s/it]



[2023-Nov-21 01:52:10]: Rescoring with AAScore complete in 479.2884!

[2023-Nov-21 01:52:10]: Combining all scores for /home/tony/DockM8/testing_single_docking/protein_protoss/rescoring_bestpose_QVINA2_clustered


Combining scores: 100%|██████████| 13/13 [00:00<00:00, 106.57files/s]


[2023-Nov-21 01:52:10]: Rescoring complete in 841.6723!


In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', rescoring_functions, 'min_max')